# Notebook de la dimensión Sede Cliente - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2 y 3. Extracción y Transformación de datos desde la base de datos - Extract and Transform

In [2]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [3]:
# Consulta SQL para extraer datos de la sede y se hace proceso de renombre
query = """
SELECT 
    s.sede_id AS id_sede,
    s.nombre AS nombre_sede,
    s.direccion AS direccion_sede,
    s.telefono,
    s.nombre_contacto,
    s.cliente_id AS id_cliente,
    s.ciudad_id AS id_ciudad
FROM sede s;
"""

In [5]:
# Cargar datos
df_sede = pd.read_sql(query, engine_fuente)
df_sede

,id_sede,nombre_sede,direccion_sede,telefono,nombre_contacto,id_cliente,id_ciudad
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,1
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,1
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,1
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,1
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,3
5,24,PASTO VITRINA /,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,4
6,25,PASTO BODEGA 29/,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,4
7,26,PALMIRA BODEGA 20 /,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,2
8,32,CLINICA FABILUX,Los angeles distrito Latino,310-70000,JUAN PEREZ,12,1
9,33,FORDES PASOANCHOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,1


In [6]:
from datetime import datetime

# Agregar fecha de carga
df_sede["fecha_carga"] = datetime.today().date()

In [7]:
# Eliminar duplicados por seguridad
df_sede = df_sede.drop_duplicates(subset=["id_sede"])

In [8]:
# Verificar resultado
print(df_sede.head())

   id_sede            nombre_sede               direccion_sede   telefono  \
0       10        FARALLONES /123  Los angeles distrito Latino  310-70000   
1       11          REMEDIOZ/ 123  Los angeles distrito Latino  310-70000   
2       13       DIME / LOS ROJOS  Los angeles distrito Latino  310-70000   
3       14  DESPACHOS / LOS ROJOS  Los angeles distrito Latino  310-70000   
4       23  POPAYAN BODEGA 28 / A  Los angeles distrito Latino  310-70000   

  nombre_contacto  id_cliente  id_ciudad fecha_carga  
0      JUAN PEREZ           4          1  2025-06-05  
1      JUAN PEREZ           4          1  2025-06-05  
2      JUAN PEREZ           4          1  2025-06-05  
3      JUAN PEREZ           4          1  2025-06-05  
4      JUAN PEREZ          11          3  2025-06-05  


## 4. Guardado de la dimensión - Load

In [9]:
# Guardar como archivo CSV
df_sede.to_csv("dim_sede_cliente.csv", index=False)

In [10]:
# Cargar en la bodega de datos OLAP
df_sede.to_sql("dim_sede_cliente", engine_bodega, if_exists="replace", index=False)

52

In [11]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_sede_cliente.csv")
print(df_verificacion.head())

   id_sede            nombre_sede               direccion_sede   telefono  \
0       10        FARALLONES /123  Los angeles distrito Latino  310-70000   
1       11          REMEDIOZ/ 123  Los angeles distrito Latino  310-70000   
2       13       DIME / LOS ROJOS  Los angeles distrito Latino  310-70000   
3       14  DESPACHOS / LOS ROJOS  Los angeles distrito Latino  310-70000   
4       23  POPAYAN BODEGA 28 / A  Los angeles distrito Latino  310-70000   

  nombre_contacto  id_cliente  id_ciudad fecha_carga  
0      JUAN PEREZ           4          1  2025-06-05  
1      JUAN PEREZ           4          1  2025-06-05  
2      JUAN PEREZ           4          1  2025-06-05  
3      JUAN PEREZ           4          1  2025-06-05  
4      JUAN PEREZ          11          3  2025-06-05  
